In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import linear_model
from sklearn import model_selection

In [2]:
conjuntoDadosSujo = pd.read_csv('DadosPrecoComputadores.csv')
conjuntoDadosSujo.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)
conjuntoDadosSujo.shape

(893, 16)

In [3]:
def transformarColunaOneHotEncode(conjuntoDados, coluna):
    classes = conjuntoDados[coluna].unique()
    dadosOneHotEncode = pd.DataFrame(0, index=range(conjuntoDados.shape[0]), columns=classes)
    for i in classes:
        dadosOneHotEncode[i] = np.where(conjuntoDados[coluna] == i, 1, 0)
    return pd.concat([conjuntoDados.drop(coluna, axis=1), dadosOneHotEncode], axis=1)

def transformarConjuntoOneHotEncode(conjuntoDados, colunas):
    for coluna in colunas:
        conjuntoDados = transformarColunaOneHotEncode(conjuntoDados, coluna)
    return conjuntoDados

def contarCategoriasemVariavel(conjuntoDados, colunas):
    for coluna in colunas:
        print(conjuntoDados[coluna].value_counts(),"\n\n")

def unificarCategorias(conjuntoDados, coluna, categorias):
    for categoria in categorias:
        conjuntoDados.loc[conjuntoDados[coluna] == categoria, coluna] = coluna+"Outros"

Podemos notar que no conjunto de dados há várias variáveis como categoria ou até mesmo texto, havendo poucos do tipo quantitativo, apenas price e spec_rating podendo tratar os outros numéricos como sendo ordinais.

In [4]:
conjuntoDadosSujo.head(5)

,brand,name,price,spec_rating,processor,CPU,Ram,Ram_type,ROM,ROM_type,GPU,display_size,resolution_width,resolution_height,OS,warranty
0,HP,Victus 15-fb0157AX Gaming Laptop,49900,73.000000,5th Gen AMD Ryzen 5 5600H,"Hexa Core, 12 Threads",8GB,DDR4,512GB,SSD,4GB AMD Radeon RX 6500M,15.6,1920.0,1080.0,Windows 11 OS,1
1,HP,15s-fq5007TU Laptop,39900,60.000000,12th Gen Intel Core i3 1215U,"Hexa Core (2P + 4E), 8 Threads",8GB,DDR4,512GB,SSD,Intel UHD Graphics,15.6,1920.0,1080.0,Windows 11 OS,1
2,Acer,One 14 Z8-415 Laptop,26990,69.323529,11th Gen Intel Core i3 1115G4,"Dual Core, 4 Threads",8GB,DDR4,512GB,SSD,Intel Iris Xe Graphics,14.0,1920.0,1080.0,Windows 11 OS,1
3,Lenovo,Yoga Slim 6 14IAP8 82WU0095IN Laptop,59729,66.000000,12th Gen Intel Core i5 1240P,"12 Cores (4P + 8E), 16 Threads",16GB,LPDDR5,512GB,SSD,Intel Integrated Iris Xe,14.0,2240.0,1400.0,Windows 11 OS,1
4,Apple,MacBook Air 2020 MGND3HN Laptop,69990,69.323529,Apple M1,Octa Core (4P + 4E),8GB,DDR4,256GB,SSD,Apple M1 Integrated Graphics,13.3,2560.0,1600.0,Mac OS,1


In [5]:
# Não há valores faltantes no conjunto de dados
conjuntoDadosSujo.isnull().sum()

brand                0
name                 0
price                0
spec_rating          0
processor            0
CPU                  0
Ram                  0
Ram_type             0
ROM                  0
ROM_type             0
GPU                  0
display_size         0
resolution_width     0
resolution_height    0
OS                   0
warranty             0
dtype: int64

In [6]:
colunasContarCategorias = ['brand', 'processor', 'CPU', 'Ram', 'Ram_type', 
                           'ROM', 'ROM_type', 'GPU', 'OS', 'display_size',
                           'resolution_width', 'resolution_height']
contarCategoriasemVariavel(conjuntoDadosSujo, colunasContarCategorias)

brand
HP           186
Lenovo       169
Asus         157
Dell         107
Acer          84
MSI           65
Samsung       28
Apple         16
Infinix       15
LG             9
Xiaomi         8
Gigabyte       8
Fujitsu        6
Zebronics      4
Ultimus        4
Tecno          3
Wings          3
Chuwi          3
Realme         3
Microsoft      2
AXL            2
Huawei         2
Honor          2
Walker         1
Vaio           1
Avita          1
Primebook      1
Ninkear        1
Razer          1
iBall          1
Name: count, dtype: int64 


processor
12th Gen Intel Core i5 1235U     49
13th Gen Intel Core i5 1335U     41
12th Gen Intel Core i3 1215U     38
11th Gen Intel Core i3 1115G4    36
12th Gen Intel Core i5 12450H    33
                                 ..
5th Gen AMD Ryzen 7  5825U        1
6th Gen AMD Ryzen 9  6900HX       1
12th Gen Intel Core i5 1230U      1
AMD Ryzen 3 7320U                 1
13th Gen ‎Intel Core i3 1315U     1
Name: count, Length: 184, dtype: int64 


CPU
Qua

Com as categorias das variáveis contadas, podemos tirar algumas conclusões:
- Na variável representando sistema operacional, tivemos duas formas diferentes para representar o mesmo item, isso acontecendo mais de uma vez, isso foi então arrumado, outra mudança feita foi definir os diferentes sistemas mac como um único.
- Boa parte das variáveis tem algumas categorias muito frequentes, enquanto o resto aparece poucas vezes. Foi colocada essa minoria em apenas uma categoria.

Vamos então usar colunas para representar cada categoria, assim tiramos algo de texto para numérico. A ideia é deixar o conjunto de dados com muita coluna e então usar um modelo simples e regularizado para que possa usar toda a informação necessária e consiga generalizar.

In [7]:
conjuntoDadosSujo.loc[conjuntoDadosSujo['OS'] == 'Windows 10  OS', 'OS'] = "Windows OS"
conjuntoDadosSujo.loc[conjuntoDadosSujo['OS'] == 'Windows 11 OS', 'OS'] = "Windows OS"
conjuntoDadosSujo.loc[conjuntoDadosSujo['OS'] == 'Windows 11  OS', 'OS'] = "Windows OS"
conjuntoDadosSujo.loc[conjuntoDadosSujo['OS'] == 'Windows 10 OS', 'OS'] = "Windows OS"
conjuntoDadosSujo.loc[conjuntoDadosSujo['OS'] == 'Mac Catalina OS', 'OS'] = "Mac OS"
conjuntoDadosSujo.loc[conjuntoDadosSujo['OS'] == 'Mac High Sierra OS', 'OS'] = "Mac OS"
conjuntoDadosSujo.loc[conjuntoDadosSujo['OS'] == 'Mac 10.15.3\t OS', 'OS'] = "Mac OS"

unificarCategorias(conjuntoDadosSujo, 'ROM', conjuntoDadosSujo['ROM'].value_counts()[6:].index)
unificarCategorias(conjuntoDadosSujo, 'brand', conjuntoDadosSujo['brand'].value_counts()[11:].index)
unificarCategorias(conjuntoDadosSujo, 'Ram_type', conjuntoDadosSujo['Ram_type'].value_counts()[7:].index)
unificarCategorias(conjuntoDadosSujo, 'CPU', conjuntoDadosSujo['CPU'].value_counts()[15:].index)
unificarCategorias(conjuntoDadosSujo, 'GPU', conjuntoDadosSujo['GPU'].value_counts()[15:].index)
unificarCategorias(conjuntoDadosSujo, 'Ram', conjuntoDadosSujo['Ram'].value_counts()[5:].index)
unificarCategorias(conjuntoDadosSujo, 'OS', conjuntoDadosSujo['OS'].value_counts()[3:].index)
unificarCategorias(conjuntoDadosSujo, 'OS', ["DOS OS"])

In [8]:
conjuntoDadosLimpo = transformarConjuntoOneHotEncode(conjuntoDadosSujo, ['brand', 'CPU', 'Ram', 'Ram_type', 'ROM', 'ROM_type', 'GPU', 'OS'])
conjuntoDadosLimpo.drop(["name", "processor"], axis=1, inplace=True)
conjuntoDadosLimpo = conjuntoDadosLimpo.sample(n=len(conjuntoDadosLimpo))
conjuntoDadosLimpo = conjuntoDadosLimpo.reset_index(drop=True)
conjuntoDadosLimpo.shape

(893, 76)

Será testado dois modelos lineares, um com penalização L1 e outro com L2. Esperamos que o modelo com regularização L1 se saia melhor por conta do contexto em que o problema está inserido. Temos um problema com muitas colunas e poucos exemplos, L1 consegue eliminar mais facilmente aquelas que não prestam e foca em treinar as interessantes, enquanto L2 busca manter muitas das variáveis, se grande parte for útil, o modelo acerta em não aliminar a maioria, mas por não ter muitos exemplos acaba não as estimando com qualidade.

In [9]:
conjuntoDadosTreino = conjuntoDadosLimpo[:700]
xTreino = conjuntoDadosTreino.drop(["price"], axis=1)
yTreino = conjuntoDadosTreino['price']
conjuntoDadosTeste = conjuntoDadosLimpo[700:]
xTeste = conjuntoDadosTeste.drop(["price"], axis=1)
yTeste = conjuntoDadosTeste['price']

regularizacaoGrid = { 
    'alpha': [1,4,10,15,20],
}

ridge = model_selection.GridSearchCV(estimator=linear_model.Ridge(), cv=10, param_grid=regularizacaoGrid, return_train_score=True)
ridge.fit(xTreino, yTreino)

lasso = model_selection.GridSearchCV(estimator=linear_model.Lasso(), cv=10, param_grid=regularizacaoGrid, return_train_score=True)
lasso.fit(xTreino, yTreino)

c:\Users\marco\Documents\Projeto Python\Nova pasta\env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.359e+09, tolerance: 2.186e+08
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=10, estimator=Lasso(), param_grid={'alpha': [1, 4, 10, 15, 20]},
             return_train_score=True)

In [10]:
ridge.cv_results_

{'mean_fit_time': array([0.01569214, 0.00743334, 0.01767871, 0.03116958, 0.0256139 ]),
 'std_fit_time': array([0.01539973, 0.00257466, 0.01150262, 0.01212455, 0.01672135]),
 'mean_score_time': array([0.00490265, 0.00375178, 0.0061321 , 0.00633907, 0.0065954 ]),
 'std_score_time': array([0.0015732 , 0.00144636, 0.00341572, 0.00229167, 0.00384889]),
 'param_alpha': masked_array(data=[1, 4, 10, 15, 20],
              mask=[False, False, False, False, False],
        fill_value=999999),
 'params': [{'alpha': 1},
  {'alpha': 4},
  {'alpha': 10},
  {'alpha': 15},
  {'alpha': 20}],
 'split0_test_score': array([0.89164839, 0.87626391, 0.85349698, 0.83894828, 0.82694331]),
 'split1_test_score': array([0.73899331, 0.6846868 , 0.64926298, 0.63402458, 0.62270574]),
 'split2_test_score': array([0.83656076, 0.84542541, 0.84741338, 0.84446091, 0.84068561]),
 'split3_test_score': array([0.84886872, 0.83209226, 0.81268309, 0.80189686, 0.79330616]),
 'split4_test_score': array([0.7049331 , 0.73378313, 0

In [11]:
lasso.cv_results_

{'mean_fit_time': array([0.13564689, 0.06182704, 0.03537736, 0.01638923, 0.03764126]),
 'std_fit_time': array([0.06524967, 0.01341898, 0.02121524, 0.00454051, 0.03007578]),
 'mean_score_time': array([0.00689518, 0.00982165, 0.00529709, 0.00379801, 0.01029429]),
 'std_score_time': array([0.00229946, 0.00561048, 0.00184575, 0.00074795, 0.00835037]),
 'param_alpha': masked_array(data=[1, 4, 10, 15, 20],
              mask=[False, False, False, False, False],
        fill_value=999999),
 'params': [{'alpha': 1},
  {'alpha': 4},
  {'alpha': 10},
  {'alpha': 15},
  {'alpha': 20}],
 'split0_test_score': array([0.89761037, 0.89721701, 0.89648796, 0.89575612, 0.89497455]),
 'split1_test_score': array([0.73999507, 0.74167033, 0.74498053, 0.74750741, 0.74971568]),
 'split2_test_score': array([0.83583712, 0.83655454, 0.8379603 , 0.83908547, 0.84014578]),
 'split3_test_score': array([0.85559528, 0.85552382, 0.85525594, 0.85484469, 0.85442361]),
 'split4_test_score': array([0.66787977, 0.67321328, 0

In [12]:
print("Resultado de lasso em teste: ",metrics.r2_score(yTeste, lasso.predict(xTeste)))
print("Resultado de ridge em teste: ",metrics.r2_score(yTeste, ridge.predict(xTeste)))

Resultado de lasso em teste:  0.8281765076145524
Resultado de ridge em teste:  0.83378377324789
